In [1]:
import tensorly as tl
from tensorly.tt_matrix import TTMatrix
from tensorly import tenalg
import torch

# Optional: Set the backend (e.g., 'numpy', 'pytorch', 'jax')
tl.set_backend('pytorch')

# 1. Create a TT-Matrix (example dimensions)
# Let's assume an input dimension of 16 and output dimension of 8
# (Original matrix would be 8x16)
# The tt_matrix factors will have appropriate shapes
input_shape = (4, 4)  # 4*4 = 16
output_shape = (2, 4) # 2*4 = 8
tt_matrix_ranks = 2 # Example ranks

# Create a random TTMatrix for demonstration
full_matrix = torch.rand(16, 8)
tt_matrix_cores = tl.decomposition.tensor_train_matrix(
    tl.reshape(full_matrix, output_shape + input_shape),
    rank=tt_matrix_ranks
)
tt_matrix = TTMatrix(tt_matrix_cores)



In [2]:
from llmcompressor.modifiers.experimental import TensorizedLinear
import torch

batch_size=32
in_features=64
out_features=64

linear = torch.nn.Linear(in_features, out_features, bias=True)

tensorized_linear = TensorizedLinear.from_linear(linear, num_cores=3, rank="same")

/Users/brian/projects/vllm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
orig_weight = linear.weight.data
tensorized_weight = tensorized_linear.to_matrix()

In [4]:
inpt = torch.rand((batch_size, in_features), requires_grad=False)
orig_output = linear(inpt)
tensorized_output = tensorized_linear(inpt)

In [7]:
(orig_output - tensorized_output).abs().max()

tensor(2.5705e-07, grad_fn=<MaxBackward1>)

In [8]:
(orig_weight - tensorized_weight).abs().max()

tensor(2.3097e-07)